<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/Alarm_Handling_NeqSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alarm Handling in NeqSim: Theory and Practice

## Overview

This tutorial covers alarm handling in NeqSim process simulations, including:

1. **Alarm Theory** - ISA-18.2 standard, alarm philosophy, and best practices
2. **Fire Detection** - Binary fire detectors with voting logic
3. **Gas Detection** - Combustible, toxic, and oxygen detectors
4. **ESD Systems** - Emergency shutdown logic and voting configurations
5. **Practical Examples** - Real-world scenarios with NeqSim code

---

## Learning Objectives

By the end of this tutorial, you will:

- ✓ Understand alarm management principles and standards
- ✓ Configure multi-level alarms (warning, high, high-high)
- ✓ Implement voting logic for spurious trip prevention
- ✓ Integrate alarms with emergency shutdown systems
- ✓ Simulate fire and gas detection scenarios
- ✓ Calculate safety system reliability

---

# Part 1: Alarm Theory and Standards

## 1.1 ISA-18.2 Alarm Management Standard

The **ISA-18.2** standard defines best practices for alarm management in process industries:

### Key Principles:

1. **Alarm Philosophy**
   - Alarms require operator response
   - Each alarm must be necessary, actionable, and unique
   - Target: <6 alarms/hour during normal operation

2. **Alarm Priority Levels**
   - **Critical**: Immediate action required (safety/environmental)
   - **High**: Prompt action required
   - **Medium**: Timely action required
   - **Low**: Awareness only

3. **Alarm States**
   - **Normal**: Process variable within limits
   - **Unacknowledged**: Alarm active, not yet acknowledged
   - **Acknowledged**: Operator aware, corrective action pending
   - **Return to Normal**: Process variable back within limits

## 1.2 Multi-Level Alarms

Process measurements often have multiple alarm thresholds:

```
             ┌─────────────────┐  HIHI Limit (Critical)
             │   DANGER ZONE   │
             ├─────────────────┤  HI Limit (Warning)
             │  WARNING ZONE   │
             ├─────────────────┤  Setpoint
   NORMAL    │  NORMAL RANGE   │
             ├─────────────────┤  LO Limit (Warning)
             │  WARNING ZONE   │
             ├─────────────────┤  LOLO Limit (Critical)
             │   DANGER ZONE   │
             └─────────────────┘
```

### Example: Gas Detector Alarms

- **20% LEL**: Warning alarm (investigate, prepare for action)
- **60% LEL**: High alarm (trigger ESD, evacuate)

## 1.3 Alarm Deadband and Delay

### Deadband
Prevents alarm chattering when process variable oscillates near limit:

$$
\text{Alarm activates at: } PV \geq \text{Limit} \\
\text{Alarm deactivates at: } PV < \text{Limit} - \text{Deadband}
$$

### Delay
Confirmation time before alarm activates (filters transients):

$$
\text{Alarm active if: } PV \geq \text{Limit for } t \geq \text{Delay}
$$

**Typical Values:**
- Process alarms: 2-5 second delay, 1-2% deadband
- Safety alarms: 1-2 second delay, 0.5-1% deadband
- Fire/gas alarms: 1-3 second delay (detector response time)

---

# Part 2: Voting Logic for Safety Systems

## 2.1 Why Use Voting Logic?

Safety systems must balance two competing requirements:

1. **Safety (Spurious Trip Prevention)**: Don't shut down on false alarms
2. **Reliability (Fail-Safe)**: Always shut down on real emergencies

**Voting logic** achieves this by requiring multiple sensors to agree before taking action.

## 2.2 Common Voting Configurations

### 1-out-of-1 (1oo1)
- **Logic**: Single sensor triggers action
- **Pros**: Simplest, most reliable for safety
- **Cons**: High spurious trip rate
- **Use**: Non-critical alarms

```
ESD = Sensor₁
```

### 2-out-of-2 (2oo2)
- **Logic**: Both sensors must trigger
- **Pros**: Low spurious trip rate
- **Cons**: Lower reliability (one sensor failure → no protection)
- **Use**: Processes where spurious trips are costly

```
ESD = Sensor₁ AND Sensor₂
```

### 2-out-of-3 (2oo3) ⭐ **RECOMMENDED**
- **Logic**: Any 2 of 3 sensors trigger
- **Pros**: Best balance of safety and reliability
- **Cons**: More expensive (3 sensors)
- **Use**: Critical safety systems (SIL 2/3)

```
ESD = (Sensor₁ AND Sensor₂) OR (Sensor₁ AND Sensor₃) OR (Sensor₂ AND Sensor₃)
```

## 2.3 Reliability Comparison

### Probability of Spurious Trip (PFD)

Assume single sensor spurious trip rate: $p = 0.01$ (1%)

| Configuration | Spurious Trip Probability | Improvement |
|---------------|--------------------------|-------------|
| **1oo1** | $p = 0.01$ | Baseline |
| **2oo2** | $p^2 = 0.0001$ | **100× better** |
| **2oo3** | $3p^2(1-p) = 0.000297$ | **34× better** |

### Probability of Dangerous Failure

Assume single sensor failure rate: $q = 0.001$ (0.1%)

| Configuration | Failure Probability | Safety |
|---------------|---------------------|--------|
| **1oo1** | $q = 0.001$ | Baseline |
| **2oo2** | $2q(1-q) ≈ 0.002$ | **2× worse** |
| **2oo3** | $3q^2 = 0.000003$ | **333× better** |

### Conclusion:
**2oo3 voting is optimal** for critical safety systems:
- 34× fewer spurious trips than 1oo1
- 333× safer than 2oo2
- Tolerates one sensor failure

---

# Part 3: Fire Detection Systems

## 3.1 Fire Detection Theory

### Types of Fire Detectors

1. **Heat Detectors**
   - Fixed temperature (e.g., 70°C)
   - Rate-of-rise (e.g., >10°C/min)

2. **Smoke Detectors**
   - Ionization (fast-flaming fires)
   - Photoelectric (slow-smoldering fires)

3. **Flame Detectors** (Oil & Gas Industry)
   - UV (ultraviolet) detectors
   - IR (infrared) detectors
   - UV/IR combination (reduces false alarms)

### Response Time

Typical fire detector response times:

- **Heat detectors**: 30-60 seconds
- **Smoke detectors**: 10-30 seconds
- **Flame detectors**: 1-5 seconds ⚡ (fastest)

## 3.2 Fire Detector Placement

**API RP 2218** guidelines:

- **Coverage**: Detectors must see 90% of protected area
- **Spacing**: 15-20m for UV/IR detectors
- **Redundancy**: Minimum 2 detectors per zone (2oo2 or 2oo3)
- **Height**: Below 15m for effective detection

### Example Layout:

```
                  SEPARATOR AREA
     ┌────────────────────────────────┐
     │                                │
     │    FD-101        FD-102        │
     │      🔥            🔥          │
     │         [SEPARATOR]            │
     │                                │
     │           FD-103               │
     │             🔥                 │
     └────────────────────────────────┘
     
     Configuration: 2-out-of-3 voting
```

---

# Part 4: Gas Detection Systems

## 4.1 Types of Gas Detectors

### 1. Combustible Gas Detectors
Measure flammable gases as **% LEL** (Lower Explosive Limit)

**Common Gases:**
- Methane (CH₄): LEL = 5.0% vol = 50,000 ppm
- Propane (C₃H₈): LEL = 2.1% vol = 21,000 ppm
- Hydrogen (H₂): LEL = 4.0% vol = 40,000 ppm

**Alarm Setpoints:**
- Warning: **20% LEL** (investigate source)
- High: **60% LEL** (evacuate, trigger ESD)

### 2. Toxic Gas Detectors
Measure harmful gases in **ppm**

**Common Gases:**
- H₂S (Hydrogen Sulfide):
  - Warning: 10 ppm (OSHA 8-hr TWA limit)
  - High: 20 ppm (STEL - Short Term Exposure Limit)
  - Danger: 100 ppm (IDLH - Immediately Dangerous)
- CO (Carbon Monoxide):
  - Warning: 35 ppm
  - High: 200 ppm

### 3. Oxygen Deficiency Detectors
Measure O₂ as **% volume**

**Alarm Setpoints:**
- Normal: 20.9% O₂
- Low: 19.5% O₂ (warning)
- Low-Low: 18.0% O₂ (evacuate)

## 4.2 LEL Calculations

### Converting ppm to %LEL

$$
\text{% LEL} = \frac{\text{Concentration (ppm)}}{\text{LEL (ppm)}} \times 100
$$

**Example:** Methane at 10,000 ppm

$$
\text{% LEL} = \frac{10{,}000}{50{,}000} \times 100 = 20\% \text{ LEL}
$$

### Explosive Range

```
100% LEL ─────────────┐
    ▲                 │
    │  EXPLOSIVE      │ UEL (Upper Explosive Limit)
 60% LEL ─────┐      │
    │         │       │
 20% LEL ────┐│       │
    │        ││ RANGE │
  0% LEL ────┴┴───────┘
    │    │   │        │
  SAFE  WARN HIGH  TOO RICH
```

## 4.3 Gas Detector Response Time

**T90 Time** = Time to reach 90% of final reading

- Catalytic bead sensors: 10-30 seconds
- Infrared sensors: 5-15 seconds
- Electrochemical (toxic): 15-60 seconds
- Oxygen sensors: 10-20 seconds

---

# Part 5: NeqSim AlarmConfig Class

## 5.1 AlarmConfig Structure

NeqSim's `AlarmConfig` class supports multi-level alarms with deadband and delay:

```java
AlarmConfig config = AlarmConfig.builder()
    .lowLowLimit(10.0)      // LOLO threshold
    .lowLimit(15.0)         // LO threshold
    .highLimit(85.0)        // HI threshold
    .highHighLimit(95.0)    // HIHI threshold
    .delay(2.0)             // Confirmation delay (seconds)
    .deadband(1.0)          // Hysteresis
    .unit("% LEL")          // Measurement unit
    .build();
```

## 5.2 Alarm Behavior

### Activation Logic

1. Process variable exceeds limit for `delay` seconds
2. Alarm transitions to ACTIVE state
3. Operator must acknowledge alarm

### Deactivation Logic

1. Process variable drops below `limit - deadband`
2. Alarm clears automatically
3. Returns to NORMAL state

### Example Timeline

```
Time    PV      Alarm State          Action
────────────────────────────────────────────
0s      50%     NORMAL               -
5s      85%     NORMAL (< delay)     -
7s      86%     ACTIVE (t > 2s)      Operator notified
10s     86%     ACKNOWLEDGED         Operator responded
15s     83%     CLEARED              PV < (85 - 1) = 84%
16s     83%     NORMAL               RTN (Return to Normal)
```

---

# Part 6: Practical Example - Fire Detection

## 6.1 Setting Up Fire Detectors

In [ ]:
%%loadFromPOM
com.equinor.neqsim:neqsim:3.0.51

In [ ]:
import neqsim.process.measurementdevice.FireDetector;
import neqsim.process.equipment.valve.BlowdownValve;
import neqsim.process.processmodel.AlarmConfig;

// Create fire detectors with location tracking
FireDetector fd1 = new FireDetector("FD-101", "Separator North");
FireDetector fd2 = new FireDetector("FD-102", "Separator South");
FireDetector fd3 = new FireDetector("FD-103", "Separator East");

// Configure alarm settings
AlarmConfig fireAlarmConfig = AlarmConfig.builder()
    .highLimit(0.5)        // Binary threshold (0=no fire, 1=fire)
    .delay(1.0)            // 1 second confirmation delay
    .unit("binary")
    .build();

fd1.setAlarmConfig(fireAlarmConfig);
fd2.setAlarmConfig(fireAlarmConfig);
fd3.setAlarmConfig(fireAlarmConfig);

System.out.println("Fire detectors configured:");
System.out.println(fd1.toString());
System.out.println(fd2.toString());
System.out.println(fd3.toString());

## 6.2 Implementing 2-out-of-3 Voting Logic

In [ ]:
// Helper method to count active fire alarms
public static int countFireAlarms(FireDetector... detectors) {
    int count = 0;
    for (FireDetector detector : detectors) {
        if (detector.isFireDetected()) {
            count++;
        }
    }
    return count;
}

// 2-out-of-3 voting logic
public static boolean check2oo3Voting(FireDetector fd1, FireDetector fd2, FireDetector fd3) {
    int activeAlarms = countFireAlarms(fd1, fd2, fd3);
    return activeAlarms >= 2;  // Trigger if ANY 2 detectors activate
}

// Test scenarios
System.out.println("\n=== TESTING 2-OUT-OF-3 VOTING LOGIC ===");

// Scenario 1: No fire
int alarms = countFireAlarms(fd1, fd2, fd3);
System.out.println("\nScenario 1: No fire detected");
System.out.println("  Active alarms: " + alarms + "/3");
System.out.println("  ESD triggered: " + (alarms >= 2 ? "YES" : "NO"));

// Scenario 2: One detector activates
fd1.detectFire();
alarms = countFireAlarms(fd1, fd2, fd3);
System.out.println("\nScenario 2: FD-101 detects fire");
System.out.println("  Active alarms: " + alarms + "/3");
System.out.println("  ESD triggered: " + (alarms >= 2 ? "YES" : "NO"));

// Scenario 3: Two detectors activate (TRIGGER!)
fd2.detectFire();
alarms = countFireAlarms(fd1, fd2, fd3);
System.out.println("\nScenario 3: FD-101 + FD-102 detect fire");
System.out.println("  Active alarms: " + alarms + "/3");
System.out.println("  ESD triggered: " + (alarms >= 2 ? "YES ⚠️" : "NO"));

// Scenario 4: All three detectors
fd3.detectFire();
alarms = countFireAlarms(fd1, fd2, fd3);
System.out.println("\nScenario 4: All detectors active");
System.out.println("  Active alarms: " + alarms + "/3");
System.out.println("  ESD triggered: " + (alarms >= 2 ? "YES ⚠️" : "NO"));

---

# Part 7: Practical Example - Gas Detection

## 7.1 Setting Up Gas Detectors

In [ ]:
import neqsim.process.measurementdevice.GasDetector;
import neqsim.process.measurementdevice.GasDetector.GasType;

// Create combustible gas detectors
GasDetector gd1 = new GasDetector("GD-101", GasType.COMBUSTIBLE, "Separator East");
GasDetector gd2 = new GasDetector("GD-102", GasType.COMBUSTIBLE, "Separator West");

// Configure for methane detection
gd1.setGasSpecies("methane");
gd1.setLowerExplosiveLimit(50000.0);  // Methane LEL = 50,000 ppm = 5% vol
gd1.setResponseTime(10.0);            // 10 second response time

gd2.setGasSpecies("methane");
gd2.setLowerExplosiveLimit(50000.0);
gd2.setResponseTime(10.0);

// Configure two-level alarms
AlarmConfig gasAlarmConfig = AlarmConfig.builder()
    .highLimit(20.0)       // 20% LEL = Warning
    .highHighLimit(60.0)   // 60% LEL = High alarm (ESD)
    .delay(2.0)            // 2 second confirmation
    .deadband(2.0)         // 2% LEL deadband
    .unit("% LEL")
    .build();

gd1.setAlarmConfig(gasAlarmConfig);
gd2.setAlarmConfig(gasAlarmConfig);

System.out.println("Gas detectors configured:");
System.out.println(gd1.toString());
System.out.println(gd2.toString());

## 7.2 Simulating Gas Leak Progression

In [ ]:
System.out.println("\n=== GAS LEAK SIMULATION ===");

// Phase 1: Normal operation
System.out.println("\nPhase 1: Normal Operation");
System.out.println("  GD-101: " + String.format("%.1f", gd1.getGasConcentration()) + "% LEL - NORMAL");
System.out.println("  GD-102: " + String.format("%.1f", gd2.getGasConcentration()) + "% LEL - NORMAL");
System.out.println("  Action: Continue normal operations");

// Phase 2: Small leak detected (warning level)
System.out.println("\nPhase 2: Gas Leak Detected");
gd1.setGasConcentration(25.0);  // 25% LEL on GD-101
gd2.setGasConcentration(15.0);  // 15% LEL on GD-102

System.out.println("  GD-101: " + String.format("%.1f", gd1.getGasConcentration()) + "% LEL - ⚠️ WARNING");
System.out.println("  GD-102: " + String.format("%.1f", gd2.getGasConcentration()) + "% LEL - NORMAL");
System.out.println("  Warning alarm: " + (gd1.isGasDetected(20.0) ? "ACTIVE" : "INACTIVE"));
System.out.println("  High alarm: " + (gd1.isHighAlarm(60.0) ? "ACTIVE" : "INACTIVE"));
System.out.println("  Action: Investigate gas source, prepare for evacuation");

// Phase 3: Large leak (high alarm, trigger ESD)
System.out.println("\nPhase 3: Large Gas Leak - EXPLOSIVE ATMOSPHERE");
gd1.setGasConcentration(65.0);  // 65% LEL
gd2.setGasConcentration(62.0);  // 62% LEL

boolean esdTrigger = gd1.isHighAlarm(60.0) && gd2.isHighAlarm(60.0);

System.out.println("  GD-101: " + String.format("%.1f", gd1.getGasConcentration()) + "% LEL - 🚨 HIGH ALARM");
System.out.println("  GD-102: " + String.format("%.1f", gd2.getGasConcentration()) + "% LEL - 🚨 HIGH ALARM");
System.out.println("  ESD Logic (2oo2): " + (esdTrigger ? "ACTIVATE ESD" : "STANDBY"));
System.out.println("  Action: " + (esdTrigger ? "EVACUATE - SHUTDOWN INITIATED" : "Monitor"));

if (esdTrigger) {
    System.out.println("\n  >>> EMERGENCY SHUTDOWN SEQUENCE <<<");
    System.out.println("  1. Sound evacuation alarm");
    System.out.println("  2. Activate blowdown valve");
    System.out.println("  3. Depressurize to flare");
    System.out.println("  4. Isolate ignition sources");
}

## 7.3 LEL Conversion Example

In [ ]:
System.out.println("\n=== LEL CONVERSION EXAMPLES ===");

// Convert ppm to %LEL
double concentration_ppm = 10000.0;  // 10,000 ppm methane
double lel_ppm = 50000.0;            // Methane LEL

GasDetector converter = new GasDetector("Converter", GasType.COMBUSTIBLE, "Lab");
converter.setLowerExplosiveLimit(lel_ppm);

double percentLEL = converter.convertPpmToPercentLEL(concentration_ppm);

System.out.println("\nMethane Concentration Conversions:");
System.out.println("  Concentration: " + String.format("%.0f ppm", concentration_ppm));
System.out.println("  LEL (methane): " + String.format("%.0f ppm", lel_ppm));
System.out.println("  Result: " + String.format("%.1f", percentLEL) + "% LEL");
System.out.println("  Status: " + (percentLEL >= 60 ? "DANGER" : percentLEL >= 20 ? "WARNING" : "SAFE"));

// Convert %LEL to ppm
double targetLEL = 20.0;  // 20% LEL
double ppm_result = converter.convertPercentLELToPpm(targetLEL);

System.out.println("\nAlarm Setpoint Calculation:");
System.out.println("  Target: " + String.format("%.0f", targetLEL) + "% LEL");
System.out.println("  Equivalent: " + String.format("%.0f ppm", ppm_result));
System.out.println("  Alarm type: Warning (investigate source)");

---

# Part 8: Combined Fire & Gas (F&G) System

## 8.1 Integrated Safety Logic

Modern process facilities use **combined Fire & Gas (F&G) detection** with multiple voting zones:

```
ESD Activation Logic:

  ESD = (Fire Zone) OR (Gas Zone)
  
  Where:
    Fire Zone = 2-out-of-3 fire detectors
    Gas Zone  = 2-out-of-2 gas high alarms (>60% LEL)
```

## 8.2 Example: Combined F&G System

In [ ]:
System.out.println("\n=== COMBINED F&G SYSTEM SIMULATION ===");

// Setup complete F&G system
FireDetector fd_a = new FireDetector("FD-201", "Zone A North");
FireDetector fd_b = new FireDetector("FD-202", "Zone A South");

GasDetector gd_a = new GasDetector("GD-201", GasType.COMBUSTIBLE, "Zone A East");
GasDetector gd_b = new GasDetector("GD-202", GasType.COMBUSTIBLE, "Zone A West");

gd_a.setGasSpecies("methane");
gd_b.setGasSpecies("methane");

System.out.println("\nF&G System Configuration:");
System.out.println("  Fire detectors: FD-201, FD-202 (2oo2)");
System.out.println("  Gas detectors: GD-201, GD-202 (2oo2 @ 60% LEL)");
System.out.println("  ESD Logic: Fire OR Gas");

// Test Scenario 1: Fire only
System.out.println("\n--- Scenario 1: Fire Event ---");
fd_a.detectFire();
fd_b.detectFire();

boolean fireESD = fd_a.isFireDetected() && fd_b.isFireDetected();
boolean gasESD = gd_a.isHighAlarm(60.0) && gd_b.isHighAlarm(60.0);
boolean esdActive = fireESD || gasESD;

System.out.println("  Fire zone: " + (fireESD ? "TRIGGERED" : "NORMAL"));
System.out.println("  Gas zone: " + (gasESD ? "TRIGGERED" : "NORMAL"));
System.out.println("  ESD Status: " + (esdActive ? "🚨 ACTIVATED (Fire)" : "NORMAL"));

// Reset
fd_a.reset();
fd_b.reset();

// Test Scenario 2: Gas leak only
System.out.println("\n--- Scenario 2: Gas Leak Event ---");
gd_a.setGasConcentration(70.0);
gd_b.setGasConcentration(65.0);

fireESD = fd_a.isFireDetected() && fd_b.isFireDetected();
gasESD = gd_a.isHighAlarm(60.0) && gd_b.isHighAlarm(60.0);
esdActive = fireESD || gasESD;

System.out.println("  Fire zone: " + (fireESD ? "TRIGGERED" : "NORMAL"));
System.out.println("  Gas zone: " + (gasESD ? "TRIGGERED" : "NORMAL"));
System.out.println("  Gas levels: GD-201 = " + gd_a.getGasConcentration() + "% LEL, GD-202 = " + gd_b.getGasConcentration() + "% LEL");
System.out.println("  ESD Status: " + (esdActive ? "🚨 ACTIVATED (Gas)" : "NORMAL"));

// Test Scenario 3: Combined event (worst case)
System.out.println("\n--- Scenario 3: Fire + Gas Event ---");
fd_a.detectFire();
fd_b.detectFire();

fireESD = fd_a.isFireDetected() && fd_b.isFireDetected();
gasESD = gd_a.isHighAlarm(60.0) && gd_b.isHighAlarm(60.0);
esdActive = fireESD || gasESD;

System.out.println("  Fire zone: " + (fireESD ? "TRIGGERED" : "NORMAL"));
System.out.println("  Gas zone: " + (gasESD ? "TRIGGERED" : "NORMAL"));
System.out.println("  ESD Status: " + (esdActive ? "🚨 ACTIVATED (Fire + Gas)" : "NORMAL"));
System.out.println("\n  ⚠️ CRITICAL EVENT - MULTIPLE HAZARDS DETECTED");
System.out.println("  Priority: Evacuate immediately, fight fire remotely only");

---

# Part 9: Safety Integrity Levels (SIL)

## 9.1 SIL Requirements (IEC 61508/61511)

Safety systems are rated by **Safety Integrity Level (SIL)**:

| SIL | PFD Range | Risk Reduction | Availability |
|-----|-----------|----------------|-------------|
| **SIL 1** | 10⁻² to 10⁻¹ | 10× to 100× | 90% - 99% |
| **SIL 2** | 10⁻³ to 10⁻² | 100× to 1,000× | 99% - 99.9% |
| **SIL 3** | 10⁻⁴ to 10⁻³ | 1,000× to 10,000× | 99.9% - 99.99% |
| **SIL 4** | 10⁻⁵ to 10⁻⁴ | 10,000× to 100,000× | 99.99% - 99.999% |

**PFD** = Probability of Failure on Demand

## 9.2 Achieving SIL with Voting

### Example Calculation:

**Given:**
- Single detector: PFD = 0.01 (1%)
- Target: SIL 2 (PFD < 0.01)

**Options:**

1. **1oo1**: PFD = 0.01 ❌ (SIL 1)
2. **2oo2**: PFD = 2 × 0.01 × (1 - 0.01) ≈ 0.0198 ❌ (worse!)
3. **2oo3**: PFD = 3 × 0.01² ≈ 0.0003 ✅ (SIL 2)

### Conclusion:
**2oo3 voting is required for SIL 2** with standard detectors

## 9.3 Proof Test Intervals

Safety systems require periodic testing:

- **SIL 1**: 12-month proof test interval
- **SIL 2**: 6-month proof test interval
- **SIL 3**: 3-month proof test interval

**Proof test** = Full functional test of safety system to detect dangerous failures

---

# Part 10: Exercises

## Exercise 1: Configure a Toxic Gas Detector

Create an H₂S detector with alarm limits:
- Warning: 10 ppm
- High: 20 ppm
- Response time: 30 seconds

**Try it yourself:**

In [ ]:
// Your solution here:
GasDetector h2sDetector = new GasDetector("H2S-101", GasType.TOXIC, "Compressor Area");

// TODO: Configure the detector
// h2sDetector.setGasSpecies(...);
// h2sDetector.setResponseTime(...);

// TODO: Create AlarmConfig with limits
// AlarmConfig h2sAlarm = AlarmConfig.builder()...

// TODO: Test with 15 ppm H2S concentration

## Exercise 2: Implement 1-out-of-2 Voting

Create a voting function that triggers ESD if **either** detector activates (1oo2).

**Try it yourself:**

In [ ]:
// Your solution here:
public static boolean check1oo2Voting(FireDetector fd1, FireDetector fd2) {
    // TODO: Implement 1oo2 logic
    // Hint: ESD activates if ANY detector triggers
    return false;  // Replace with your logic
}

// Test your function
FireDetector test1 = new FireDetector("TEST-1", "Zone 1");
FireDetector test2 = new FireDetector("TEST-2", "Zone 2");

test1.detectFire();
System.out.println("1oo2 Result: " + check1oo2Voting(test1, test2));

## Exercise 3: Calculate Spurious Trip Rate

Given:
- Single detector spurious trip rate: p = 0.02 (2%)
- Configuration: 2-out-of-3 voting

Calculate the system spurious trip probability.

**Formula:** $P_{2oo3} = 3p^2(1-p)$

**Try it yourself:**

In [ ]:
// Your solution here:
double p = 0.02;  // 2% spurious trip rate per detector

// TODO: Calculate 2oo3 spurious trip probability
// double p_2oo3 = ...;

// TODO: Calculate improvement factor
// double improvement = p / p_2oo3;

// System.out.println("2oo3 Spurious Trip Rate: " + p_2oo3);
// System.out.println("Improvement: " + improvement + "x better");

---

# Part 11: Summary and Best Practices

## 11.1 Key Takeaways

✅ **Alarm Management**
- Follow ISA-18.2 guidelines
- Keep alarm rates < 6/hour during normal operation
- Use multi-level alarms (warning → high → critical)
- Configure appropriate deadband and delay

✅ **Fire Detection**
- Use UV/IR flame detectors for rapid response
- Implement 2oo3 voting for critical areas
- Ensure proper detector coverage (90% of area)
- Test detectors quarterly

✅ **Gas Detection**
- Set warning at 20% LEL, high alarm at 60% LEL
- Use 2oo2 or 2oo3 voting to prevent spurious trips
- Account for detector response time (T90)
- Calibrate monthly with certified test gas

✅ **ESD Systems**
- Combine fire and gas detection (F&G systems)
- Use latching logic (manual reset required)
- Design for SIL 2 or SIL 3 depending on risk
- Proof test every 6 months (SIL 2) or 3 months (SIL 3)

## 11.2 Common Mistakes to Avoid

❌ **Don't:**
- Use 1oo1 voting for critical safety functions
- Set alarm limits too tight (causes nuisance alarms)
- Ignore deadband (causes alarm chattering)
- Skip proof testing (dangerous failures go undetected)

✅ **Do:**
- Use 2oo3 voting for SIL 2/3 applications
- Rationalize alarms before implementation
- Document alarm setpoints and philosophy
- Train operators on alarm response procedures

## 11.3 Further Reading

**Standards:**
- ISA-18.2: Management of Alarm Systems for the Process Industries
- IEC 61508: Functional Safety of Electrical/Electronic Systems
- IEC 61511: Safety Instrumented Systems for Process Industry
- API RP 2218: Fireproofing Practices in Petroleum Facilities

**NeqSim Documentation:**
- Process Control Guide: `docs/wiki/process_control.md`
- ESD Fire Alarm System: `docs/esd_fire_alarm_system.md`
- Measurement Devices: `src/main/java/neqsim/process/measurementdevice/`

---
